In [1]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn.ensemble import RandomForestRegressor
from sklearn.cross_validation import train_test_split
from sklearn.metrics import mean_squared_error,mean_absolute_error
from sklearn.linear_model import Lasso, LinearRegression
from copy import deepcopy
import os
from sklearn.preprocessing import LabelEncoder, OneHotEncoder

%matplotlib inline

In [2]:
loc = '/Users/elena/Documents/Kaggle/Allstate/data/'
train = pd.read_csv(os.path.join(loc,'train.csv'))
train.drop('id',axis=1,inplace=True)

test = pd.read_csv(os.path.join(loc,'test.csv'))
ids = test['id']
test.drop('id',axis=1,inplace=True)

submission = pd.read_csv(os.path.join(loc,"sample_submission.csv"))

In [3]:
# List of categorical columns
keepcats = [col for col in train.columns if 'cat' in col]
# List of continuous columns
keepcont = [col for col in train.columns if 'cont' in col]

print 'Categorical: ',len(keepcats)
print 'Continuous: ',len(keepcont)

Categorical:  116
Continuous:  14


In [4]:
newlist = deepcopy(keepcont)
newlist.remove("cont1")
newlist.remove("cont11")
newlist.remove("cont6")
newlist.append("loss")

dataset = train[newlist]
dataset['loss']=np.log1p(dataset['loss'])
dataset.rename(columns={'loss':'log-loss'},inplace=True)
dataset.head()

/Users/elena/anaconda/lib/python2.7/site-packages/ipykernel/__main__.py:8: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
/Users/elena/anaconda/lib/python2.7/site-packages/pandas/core/frame.py:2754: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  **kwargs)


,cont2,cont3,cont4,cont5,cont7,cont8,cont9,cont10,cont12,cont13,cont14,log-loss
0,0.245921,0.187583,0.789639,0.310061,0.335060,0.30260,0.67135,0.83510,0.594646,0.822493,0.714843,7.702637
1,0.737068,0.592681,0.614134,0.885834,0.436585,0.60087,0.35127,0.43919,0.366307,0.611431,0.304496,7.158203
2,0.358319,0.484196,0.236924,0.397069,0.315545,0.27320,0.26076,0.32446,0.373424,0.195709,0.774425,8.008396
3,0.555782,0.527991,0.373816,0.422268,0.391128,0.31796,0.32128,0.44467,0.321570,0.605077,0.602642,6.846784
4,0.159990,0.527991,0.473202,0.704268,0.247408,0.24564,0.22089,0.21230,0.202213,0.246011,0.432606,7.924742


In [5]:
## Now let's code up the categorical data (0's and 1's instead of ABC's)
labels = []
cols = train.columns
newlabels = []
for i in range(0,116):
    trainer = train[cols[i]].unique()
    tester = test[cols[i]].unique()
    # We want the intersection of labels from the training and testing datasets
    # in case there are labels in the test dataset that don't show up in the training dataset.
    labels.append(list(set(trainer) | set(tester)))    
    
    for thing in labels[-1]:
        newlabels.append(cols[i]+'-'+str(thing))

#One hot encode all categorical attributes
cats = []
for i in range(0, 116):
    #Label encode
    label_encoder = LabelEncoder()
    label_encoder.fit(labels[i])
    feature = label_encoder.transform(train.iloc[:,i])
    feature = feature.reshape(train.shape[0], 1)
    
    
    #One hot encode
    onehot_encoder = OneHotEncoder(sparse=False,n_values=len(labels[i]))
    feature = onehot_encoder.fit_transform(feature)
    cats.append(feature)

# Make a 2D array from a list of 1D arrays
encoded_cats = np.column_stack(cats)
print len(newlabels)
print encoded_cats.shape

1176
(188318, 1176)


In [6]:
# Based on analysis done in Kaggle forum, get rid of a couple highly
# correlated categorical variables:
get_rid_of = list(['cat2-','cat6-','cat8-','cat7-','cat16-'])
new_cat_list = []

for lab in newlabels:
    ans = 'yes'
    for bad in get_rid_of:
        if bad in lab:
            ans = 'no'
            
    if ans=='yes':
        new_cat_list.append(lab)

print len(newlabels),len(new_cat_list)

inds = [i for i in range(len(newlabels)) if newlabels[i] in new_cat_list]
df = pd.DataFrame(encoded_cats[:,inds],columns=new_cat_list)

1176 1166


In [7]:
final_data = pd.concat([dataset,df],axis=1)
final_data.head()

,cont2,cont3,cont4,cont5,cont7,cont8,cont9,cont10,cont12,cont13,...,cat116-AP,cat116-AS,cat116-AR,cat116-AU,cat116-AT,cat116-AW,cat116-AV,cat116-AY,cat116-AX,cat116-N
0,0.245921,0.187583,0.789639,0.310061,0.335060,0.30260,0.67135,0.83510,0.594646,0.822493,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
1,0.737068,0.592681,0.614134,0.885834,0.436585,0.60087,0.35127,0.43919,0.366307,0.611431,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
2,0.358319,0.484196,0.236924,0.397069,0.315545,0.27320,0.26076,0.32446,0.373424,0.195709,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
3,0.555782,0.527991,0.373816,0.422268,0.391128,0.31796,0.32128,0.44467,0.321570,0.605077,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
4,0.159990,0.527991,0.473202,0.704268,0.247408,0.24564,0.22089,0.21230,0.202213,0.246011,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


In [8]:
y = final_data['log-loss']
x = final_data.drop('log-loss',axis=1)

In [10]:
X_train, X_test, y_train, y_test = train_test_split(x, y, test_size=.3)

newmodel = LinearRegression(normalize=True)
newmodel.fit(X_train,y_train)

#print np.min(np.exp(y_test)),np.max(np.exp(y_test))

pred = newmodel.predict(X_test)
# pred[pred<0.0]=np.nan
# pred[pred>100]=np.nan

print mean_absolute_error(np.expm1(y_test),np.expm1(pred))

ValueError: Input contains NaN, infinity or a value too large for dtype('float64').

In [ ]:
plt.scatter(y_test,newmodel.predict(X_test))
plt.ylabel('Model predicted')
plt.xlabel('Truth')
plt.xlim([0,14])
plt.ylim([0,14])

In [ ]:
plt.scatter(y_test,newmodel.predict(X_test))